In [1]:
import ibm_db
#accessing database details from service credential and creating dsn string
dsn_hostname = "dashdb-txn-sbox-yp-dal09-14.services.dal.bluemix.net" 
dsn_uid = "drb56509"       
dsn_pwd = "0sqr0nk34fd-wtn6"      

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"            
dsn_port = "50000"                
dsn_protocol = "TCPIP"  

dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)

#print the connection string to check correct values are specified
print(dsn)

DRIVER={IBM DB2 ODBC DRIVER};DATABASE=BLUDB;HOSTNAME=dashdb-txn-sbox-yp-dal09-14.services.dal.bluemix.net;PORT=50000;PROTOCOL=TCPIP;UID=drb56509;PWD=0sqr0nk34fd-wtn6;


In [2]:
#establishing the connection to the database
try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )

Connected to database:  BLUDB as user:  drb56509 on host:  dashdb-txn-sbox-yp-dal09-14.services.dal.bluemix.net


In [5]:
server = ibm_db.server_info(conn)
print("Server details")
print ("DBMS_NAME: ", server.DBMS_NAME)
print ("DBMS_VER:  ", server.DBMS_VER)
print ("DB_NAME:   ", server.DB_NAME)
print("   ")
client = ibm_db.client_info(conn)
print("Client details")
print(" ")
print ("DRIVER_NAME:          ", client.DRIVER_NAME) 
print ("DRIVER_VER:           ", client.DRIVER_VER)
print ("DATA_SOURCE_NAME:     ", client.DATA_SOURCE_NAME)
print ("DRIVER_ODBC_VER:      ", client.DRIVER_ODBC_VER)
print ("ODBC_VER:             ", client.ODBC_VER)
print ("ODBC_SQL_CONFORMANCE: ", client.ODBC_SQL_CONFORMANCE)
print ("APPL_CODEPAGE:        ", client.APPL_CODEPAGE)
print ("CONN_CODEPAGE:        ", client.CONN_CODEPAGE)

Server details
DBMS_NAME:  DB2/LINUXX8664
DBMS_VER:   11.01.0404
DB_NAME:    BLUDB
   
Client details
 
DRIVER_NAME:           libdb2.a
DRIVER_VER:            11.05.0500
DATA_SOURCE_NAME:      BLUDB
DRIVER_ODBC_VER:       03.51
ODBC_VER:              03.01.0000
ODBC_SQL_CONFORMANCE:  EXTENDED
APPL_CODEPAGE:         1208
CONN_CODEPAGE:         1208


In [16]:
#Lets first drop the table INSTRUCTOR in case it exists from a previous attempt
dropQuery = "drop table INSTRUCTOR"

#Now execute the drop statment
dropStmt = ibm_db.exec_immediate(conn, dropQuery)

# ignore if error comes once as table may not be already created in database
createQuery = "create table INSTRUCTOR(id INTEGER PRIMARY KEY NOT NULL,\
            fname VARCHAR(20),lname varchar(20),city varchar(20),city_code char(2))"
createStmt = ibm_db.exec_immediate(conn, createQuery)
# adding values to the table
insertQuery = "insert into INSTRUCTOR(id,fname,lname,city,city_code) \
               VALUES (1,'Vishal','Pundir','Roorkee','RK'),\
                      (2,'Arun','Pundir','Haridwar','Hd'),\
                      (3, 'Sushil', 'Pundir', 'uttarakhand', 'Uk')"
insertStmt = ibm_db.exec_immediate(conn, insertQuery)

In [18]:
selectQuery = "select * from INSTRUCTOR"
#Execute the statement
selectStmt = ibm_db.exec_immediate(conn, selectQuery)
#Fetch the Dictionary (for the id and first name of each row 
while ibm_db.fetch_row(selectStmt) != False:
   print (" ID:",  ibm_db.result(selectStmt, 0), " FNAME:",  ibm_db.result(selectStmt, "FNAME"))

 ID: 1  FNAME: Vishal
 ID: 2  FNAME: Arun
 ID: 3  FNAME: Sushil


In [19]:
# we can use update query also to update anything in our table
updatequery= "UPDATE INSTRUCTOR set city='Mohali' where fname='Vishal'"
updatestmtn=ibm_db.exec_immediate(conn,updatequery)

In [20]:
# Now retrieving table into pandas as dataframe
import pandas
import ibm_db_dbi

In [21]:
#connection for pandas
pconn = ibm_db_dbi.Connection(conn)

In [22]:
#query statement to retrieve all rows in INSTRUCTOR table
selectQuery = "select * from INSTRUCTOR"
#retrieve the query results into a pandas dataframe
pdf = pandas.read_sql(selectQuery, pconn)
pdf

,ID,FNAME,LNAME,CITY,CITY_CODE
0,1,Vishal,Pundir,Mohali,RK
1,2,Arun,Pundir,Haridwar,Hd
2,3,Sushil,Pundir,uttarakhand,Uk


In [23]:
ibm_db.close(conn)

True